# CSC_52081_EP Project

Advanced Machine Learning and Autonomous Agents Project

## Introduction

Reinforcement Learning (RL) has emerged as a robust framework for training autonomous agents to learn optimal behaviors through environmental interactions. This study utilizes the [`CarRacing-v3`](https://gymnasium.farama.org/environments/box2d/car_racing/) environment from Gymnasium, which presents a challenging control task in a racing scenario.

### Environment

The environment features a high-dimensional observation space, represented by a $96 \times 96$ RGB image capturing the car and track, necessitating the use of deep convolutional neural networks (CNNs) for effective feature extraction.

#### Action Space

The action space in CarRacing-v3 supports both continuous and discrete control modes.

In **continuous mode**, the agent outputs three real-valued commands:

- steering (ranging from $-1$ to $+1$)
- gas
- braking

In **discrete mode**, the action space is simplified to five actions:

- do nothing
- steer left
- steer right
- gas
- brake

This dual action representation enables a comprehensive evaluation of various RL algorithms under different control settings.

#### Reward

The reward structure combines a penalty of $-0.1$ per frame and a reward of $+\frac{1000}{N}$ for each new track tile visited, where $N$ is the total number of tiles. This incentivizes the agent to balance exploration (visiting tiles) with efficiency (minimizing frame usage). For example, completing the race after visiting all $N$ tiles in 732 frames yields a reward of $1000 - 0.1 \times 732 = 926.8$ points.

### Objective

The primary objective of this project is to compare RL policies across discrete and continuous action modalities. For discrete control, methods like **Deep Q-Network** (DQN) and **SARSA** are implemented, while continuous control is explored using approaches such as the **Cross-Entropy Method** (CEM), **Self-Adaptive Evolution Strategy** (SA-ES), and policy gradient techniques like **Proximal Policy Optimization** (PPO) and **Soft Actor-Critic** (SAC). This comparative analysis aims to understand the strengths and limitations of each method in handling complex decision spaces.

The high-dimensional visual inputs in `CarRacing-v3` require effective feature extraction, addressed through a tailored CNN architecture. Transitioning between discrete and continuous action representations also demands careful algorithmic design and parameter tuning to ensure stable learning and convergence. While prior studies have often focused on either discrete or continuous action spaces separately, this work adopts a comparative approach, evaluating different agents within the same environment to assess performance under similar conditions.

At this stage, the work outlines the methodology and anticipated challenges, focusing on designing the CNN-based feature extractor, implementing RL algorithms, and establishing a framework for performance comparison. Preliminary findings are yet to be finalized, but the study is expected to provide insights into applying RL in high-dimensional, real-time control tasks. Limitations include the preliminary nature of experiments and the need for further tuning and validation. Future work will involve extensive empirical evaluations, exploring additional policy gradient methods, and refining the network architecture to better handle the complexities of `CarRacing-v3`.

### GitHub

The project's code is available on [GitHub](https://github.com/tr0fin0/ensta_CSC_52081_EP_project), offering a reproducible framework for future investigations and extensions.

## Installation

### Environment

#### WSL, Linux or MacOS

A `Python Virtual Environment` will be used for this project by run the following on a terminal on the project folder:

```bash
sudo apt install python3.10-venv
python3 -m venv env
source env/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install -r requirements.txt
```

### Imports

In [ ]:
import datetime
import gymnasium as gym
import gymnasium.wrappers as gym_wrap
import torch
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

plt.ion()
sns.set_context("talk")


from IPython.display import Video
from pathlib import Path
from typing import List


DIRECTORY_OUTPUT = "output"
DIRECTORY_MODELS = Path(f"{DIRECTORY_OUTPUT}/models/")
DIRECTORY_FIGURES = Path(f"{DIRECTORY_OUTPUT}/images/")
DIRECTORY_LOGS = Path(f"{DIRECTORY_OUTPUT}/logs/")

for directory in [DIRECTORY_MODELS, DIRECTORY_FIGURES, DIRECTORY_LOGS]:
    if not directory.exists():
        directory.mkdir(parents=True)

print(DIRECTORY_OUTPUT)
print(DIRECTORY_MODELS)
print(DIRECTORY_FIGURES)
print(DIRECTORY_LOGS)

## Demonstration

In [ ]:
VIDEO_DEMO = "CSC_52081_EP_demonstration"
(DIRECTORY_FIGURES / f"{VIDEO_DEMO}.mp4").unlink(missing_ok=True)


env = gym.make(
    "CarRacing-v3",
    render_mode="rgb_array",
    lap_complete_percent=0.95,
    domain_randomize=False,
    continuous=False
)
env = gym.wrappers.RecordVideo(env, video_folder=str(DIRECTORY_FIGURES), name_prefix=VIDEO_DEMO)


done = False
observation, info = env.reset()

while not done:
    action = env.action_space.sample()

    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

env.close()


Video(
    DIRECTORY_FIGURES / f"{VIDEO_DEMO}-episode-0.mp4",
    embed=True,
    html_attributes="controls autoplay loop",
)

## Description

only demonstration is right. from below here is only experimental.

### Global Definitions

### SAC

In [ ]:
import datetime
import gymnasium as gym
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.ion()
sns.set_context("talk")

DIRECTORY_OUTPUT = "output"
DIRECTORY_MODELS = Path(f"{DIRECTORY_OUTPUT}/models/")
DIRECTORY_FIGURES = Path(f"{DIRECTORY_OUTPUT}/images/")
DIRECTORY_LOGS = Path(f"{DIRECTORY_OUTPUT}/logs/")

for directory in [DIRECTORY_MODELS, DIRECTORY_FIGURES, DIRECTORY_LOGS]:
    if not directory.exists():
        directory.mkdir(parents=True)


from Agent_SAC import AgentSAC
from CNN import CNN
from utils import get_environment, plot_reward
import datetime
import numpy as np
import torch
import matplotlib.pyplot as plt

MODEL_LOAD = False
MODEL_ID = 400000

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCHES = 64
EPISODES = 3000
EPISODES_EXTRA = 300

# Use a continuous environment for SAC
env_continuous = get_environment(is_continuous=True)
state, info = env_continuous.reset()

agent = AgentSAC(
    state.shape,
    env_continuous.action_space.shape,
    DEVICE,
    DIRECTORY_MODELS,
    DIRECTORY_LOGS,
    CNN,
    policy_type="Gaussian",  # or "Deterministic" if preferred
    gamma=0.99,
    tau=0.005,
    lr=3e-5,
    alpha=0.2,
    auto_alpha_tuning=True,
    batch_size=BATCHES,
    hidden_size=256,
    target_update_interval=1,
    load_state=MODEL_LOAD,
    load_model=f'SAC_{MODEL_ID}.pt' if MODEL_LOAD else None,
)

episode = EPISODES if MODEL_LOAD else 0
actions = MODEL_ID if MODEL_LOAD else 0
episode_rewards = []
episode_lengths = []
episode_losses = []
episode_dates = []
episode_times = []

interval_learn = 4      # [actions]
interval_save = 10000   # [actions]
interval_log = 10       # [episodes]

while episode <= EPISODES + (EPISODES_EXTRA if MODEL_LOAD else 0):
    episode += 1
    episode_reward = 0
    episode_length = 0

    done = False
    losses = []

    while not done:
        actions += 1
        episode_length += 1

        action = agent.get_action(state)
        if np.isnan(action).any():
            action = env_continuous.action_space.sample()  # Sample a random action if the action is NaN
        next_state, reward, terminated, truncated, info = env_continuous.step(action)

        # print(action, terminated, truncated)

        episode_reward += reward
        agent.store(state, action, reward, next_state, terminated)

        state = next_state
        done = terminated or truncated

        if actions % interval_learn == 0:
            qf1_loss, qf2_loss, policy_loss, alpha_loss = agent.update_network(BATCHES)
            losses.append((qf1_loss, qf2_loss, policy_loss, alpha_loss))

        if actions % interval_save == 0:
            agent.save()

    state, info = env_continuous.reset()

    episode_rewards.append(episode_reward)
    episode_lengths.append(episode_length)
    episode_losses.append(np.mean(losses, axis=0))
    now_time = datetime.datetime.now()
    episode_dates.append(now_time.date().strftime('%Y-%m-%d'))
    episode_times.append(now_time.time().strftime('%H:%M:%S'))

    plot_reward(episode, episode_rewards, actions)

    if episode % interval_log == 0:
        agent.write_log(
            episode_dates,
            episode_times,
            episode_rewards,
            episode_lengths,
            episode_losses,
        )

agent.save()
agent.write_log(
    episode_dates,
    episode_times,
    episode_rewards,
    episode_lengths,
    episode_losses,
)
env_continuous.close()

plt.ioff()
plt.show()

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID = 2500
SEEDS = [i for i in range(2)]


env_continuous = get_environment_continuous()
state, info = env_continuous.reset()

agent = Agent_SAC(
    state.shape,
    env_continuous.action_space,
    DEVICE,
    DIRECTORY_MODELS,
    DIRECTORY_LOGS,
    load_state='eval',
    load_model= f'SAC_{MODEL_ID}.pt',
)

agent_evaluation(agent, env_continuous, SEEDS)